In [1]:
import cv2
import numpy as np
from skimage.filters import threshold_local
import tensorflow as tf
from skimage import measure
import imutils

In [3]:
def sortare_contur(contur_caracter):
    """
   Sortam conturul de la stanga la dreapta
    """
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in contur_caracter]
    (contur_caracter, boundingBoxes) = zip(*sorted(zip(contur_caracter, boundingBoxes),
                                                      key=lambda b: b[1][i], reverse=False))
    return contur_caracter


In [4]:
def segment_chars(plate_img, fixed_width):
    V = cv2.split(cv2.cvtColor(plate_img, cv2.COLOR_BGR2HSV))[2]
    T = threshold_local(V, 29, offset=15, method='gaussian')
    thresh = (V > T).astype('uint8') * 255
    thresh = cv2.bitwise_not(thresh)

    # redimensionarea placutei la o dimensiune canonica
    plate_img = imutils.resize(plate_img, width=fixed_width)
    thresh = imutils.resize(thresh, width=fixed_width)
    bgr_thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

    # analizam componentele conectate si initializam mask-ul pentru a stoca locatiile
    # a caracterelor
    labels = measure.label(thresh, neighbors=8, background=0)
    charCandidates = np.zeros(thresh.shape, dtype='uint8')

    # facem un loop peste caracterele unice
    character = []
    for label in np.unique(labels):
        # daca este caracter de fundal,il ignoram
        if label == 0:
            continue
        #in caz contrar,facem mask label-ul pentru a afisa doar caracterele conectate
        # in label-ul curent,apoi gasim contururile in label mask
        labelMask = np.zeros(thresh.shape, dtype='uint8')
        labelMask[labels == label] = 255
        cnts = cv2.findContours(labelMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]

        # ne asiguram ca cel putin un contur o fost gasit
        if len(cnts) > 0:

            #luam cel mai mare contur  care corespunde label mask-ului apoi
            # luam bound box-ul conturului
            c = max(cnts, key=cv2.contourArea)
            (boxX, boxY, boxW, boxH) = cv2.boundingRect(c)

            # calculam aspectul , soliditatea  si inaltimea componentei
            aspect = boxW / float(boxH)
            soliditate = cv2.contourArea(c) / float(boxW * boxH)
            inaltime = boxH / float(plate_img.shape[0])

            # determinam daca aspectul , soliditatea  si inaltimea componentei 
            # trec regulile testului
            tinemAspect = aspect < 1.0
            tinemSoliditate = soliditate > 0.15
            tinemInaltime = inaltime > 0.5 and inaltime < 0.95

            #verificam daca cele 3 componente trec testul
            if tinemAspect and tinemSoliditate and tinemInaltime and boxW > 14:
                # calculam 'convex hull mask' si il desenam pe character mask
                hull = cv2.convexHull(c)
                cv2.drawContours(charCandidates, [hull], -1, 255, -1)
    _, contours, hier = cv2.findContours(charCandidates, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        contours = sortare_contur(contours)
        addPixel = 4  #valoarea se va adauga pentru fiecare dimensiune a caracterului
        for c in contours:
            (x, y, w, h) = cv2.boundingRect(c)
            if y > addPixel:
                y = y - addPixel
            else:
                y = 0
            if x > addPixel:
                x = x - addPixel
            else:
                x = 0
            temp = bgr_thresh[y:y + h + (addPixel * 2), x:x + w + (addPixel * 2)]
            characters.append(temp)
        return characters
    else:
        return None
 

In [25]:
class PlateFinder:
    def __init__(self):
        self.min_area = 4500  # aria minima a placutei de inmatriculare
        self.max_area = 30000  # aria maxima a placutei de inmatriculare
        self.element_structure = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(22, 3))
    def preprocess(self, input_img):
        imgBlurred = cv2.GaussianBlur(input_img, (7, 7), 0) 
        gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)  # convertim la gri
        sobelx = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize=3)  # sobelx pentru a gasi marginile verticale
        ret2, threshold_img = cv2.threshold(sobelx, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        element = self.element_structure
        morph_n_thresholded_img = threshold_img.copy()
        cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_n_thresholded_img)
        return morph_n_thresholded_img
    def extract_contours(self, after_preprocess):
        _, contours, _ = cv2.findContours(after_preprocess, mode=cv2.RETR_EXTERNAL,
                                                    method=cv2.CHAIN_APPROX_NONE)
        return contours
    def clean_plate(self, plate):
        gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        _, contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if contours:
            areas = [cv2.contourArea(c) for c in contours]
            max_index = np.argmax(areas)  # index al celui mai mare contur din zona
            max_cnt = contours[max_index]
            max_cntArea = areas[max_index]
            x, y, w, h = cv2.boundingRect(max_cnt)
            rect = cv2.minAreaRect(max_cnt)
            rotatedPlate = plate
            if not self.ratioCheck(max_cntArea, rotatedPlate.shape[1], rotatedPlate.shape[0]):
                return plate, False, None
            return rotatedPlate, True, [x, y, w, h]
        else:
            return plate, False, None

In [26]:
    def check_plate(self, input_img, contour):
        min_rect = cv2.minAreaRect(contour)
        if self.validateRatio(min_rect):
            x, y, w, h = cv2.boundingRect(contour)
            after_validation_img = input_img[y:y + h, x:x + w]
            after_clean_plate_img, plateFound, coordinates = self.clean_plate(after_validation_img)
            if plateFound:
                characters_on_plate = self.find_characters_on_plate(after_clean_plate_img)
                if (characters_on_plate is not None and len(characters_on_plate) == 8):
                    x1, y1, w1, h1 = coordinates
                    coordinates = x1 + x, y1 + y
                    after_check_plate_img = after_clean_plate_img
                    return after_check_plate_img, characters_on_plate, coordinates
        return None, None, None

In [27]:
    def gaseste_placutele_posibile(self, input_img):
        """
        Cautam toate contururile care pot fi placute
        """
        placute = []
        self.char_on_plate = []
        self.corresponding_area = []
        self.after_preprocess = self.preprocess(input_img)
        possible_plate_contours = self.extract_contours(self.after_preprocess)
        for cnts in possible_plate_contours:
            placute, characters_on_plate, coordinates = self.check_plate(input_img, cnts)
            if placute is not None:
                placute.append(plate)
                self.char_on_plate.append(characters_on_plate)
                self.corresponding_area.append(coordinates)
        if (len(placute) > 0):
            return placute
        else:
            return None
    def cautare_caractere_placuta(self, placute):
        caractereGasite = segment_chars(placute, 400)
        if caractereGasite:
            return caractereGasite

    # PLATE FEATURES
    def ratioCheck(self, area, width, height):
        min = self.min_area
        max = self.max_area
        ratioMin = 3
        ratioMax = 6
        ratio = float(width) / float(height)
        if ratio < 1:
            ratio = 1 / ratio
        if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
            return False
        return True
    def preRatioCheck(self, area, width, height):
        min = self.min_area
        max = self.max_area
        ratioMin = 2.5
        ratioMax = 7
        ratio = float(width) / float(height)
        if ratio < 1:
            ratio = 1 / ratio
        if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
            return False
        return True
    def validateRatio(self, rect):
        (x, y), (width, height), rect_angle = rect
        if (width > height):
            angle = -rect_angle
        else:
            angle = 90 + rect_angle
        if angle > 15:
            return False
        if (height == 0 or width == 0):
            return False
        area = width * height
        if not self.preRatioCheck(area, width, height):
            return False
        else:
            return True


In [28]:
class NeuralNetwork:
    def __init__(self):
        self.model_file = "C:/Users/Bogdan/Desktop/Car-License-Plate-Recognition-master/Car-License-Plate-Recognition-master/model/binary_128_0.50_ver3.pb"
        self.label_file = "C:/Users/Bogdan/Desktop/Car-License-Plate-Recognition-master/Car-License-Plate-Recognition-master/model/binary_128_0.50_labels_ver2.txt"
        self.label = self.load_label(self.label_file)
        self.graph = self.load_graph(self.model_file)
        self.sess = tf.compat.v1.Session(graph=self.graph)
    def load_graph(self, modelFile):
        graph = tf.Graph()
        graph_def = tf.compat.v1.GraphDef()
        with open(modelFile, "rb") as f:
            graph_def.ParseFromString(f.read())
        with graph.as_default():
            tf.import_graph_def(graph_def)
        return graph
    def load_label(self, labelFile):
        label = []
        proto_as_ascii_lines = tf.io.gfile.GFile(labelFile).readlines()
        for l in proto_as_ascii_lines:
            label.append(l.rstrip())
        return label
    def convert_tensor(self, image, imageSizeOuput):
        """
    luam o imagine si o facem in tensor
    """
        image = cv2.resize(image, dsize=(imageSizeOuput, imageSizeOuput), interpolation=cv2.INTER_CUBIC)
        np_image_data = np.asarray(image)
        np_image_data = cv2.normalize(np_image_data.astype('float'), None, -0.5, .5, cv2.NORM_MINMAX)
        np_final = np.expand_dims(np_image_data, axis=0)
        return np_final
    def label_image(self, tensor):
        input_name = "import/input"
        output_name = "import/final_result"
        input_operation = self.graph.get_operation_by_name(input_name)
        output_operation = self.graph.get_operation_by_name(output_name)
        results = self.sess.run(output_operation.outputs[0],
                                {input_operation.outputs[0]: tensor})
        results = np.squeeze(results)
        labels = self.label
        top = results.argsort()[-1:][::-1]
        return labels[top[0]]
    def label_image_list(self, listImages, imageSizeOuput):
        plate = ""
        for img in listImages:
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            plate = plate + self.label_image(self.convert_tensor(img, imageSizeOuput))
        return plate, len(plate)

In [30]:
if __name__ == "__main__":
    findPlate = PlateFinder()

    # Initializam 'NeuralNetwork'
    model = NeuralNetwork()

    cap = cv2.VideoCapture('C:/video.MOV')
    while (cap.isOpened()):
        ret, img = cap.read()
        if ret == True:
            cv2.imshow('original video', img)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            # cv2.waitKey(0)
            possible_plates = findPlate.gaseste_placutele_posibile(img)
            if possible_plates is not None:
                for i, p in enumerate(possible_plates):
                    chars_on_plate = findPlate.char_on_plate[i]
                    recognized_plate, _ = model.label_image_list(chars_on_plate, imageSizeOuput=128)
                    print(recognized_plate)
                    cv2.imshow('plate', p)
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        break


        else:
            break
    cap.release()
    cv2.destroyAllWindows()

AttributeError: 'PlateFinder' object has no attribute 'gaseste_placutele_posibile'